# RQ1 - Eyetracking Fixation Metrics

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import utils.GenSnippetsLib as gsl
import os

In [2]:
screen_resolution = (1920, 1080)

## Import Eyetracking Data

In [3]:
df_query = pd.read_csv("./data/filteredData/filtered_data.csv")
df_eyetracking_events = pd.DataFrame(columns=["Participant", "Algorithm", "Path"])
snippets = df_query["Algorithm"].unique()
participants = df_query["Participant"].unique()
for participant in participants:
    for snippet in snippets:
        path = f"./data/filteredData/Participant{str(participant).zfill(2)}/{snippet}_Code_eyetracking.csv"
        # check if path exists
        if os.path.exists(path):
            df_eyetracking_events.loc[len(df_eyetracking_events)] = [participant, snippet, path]
df_eyetracking_events

,Participant,Algorithm,Path
0,1,IsPrime,./data/filteredData/Participant01/IsPrime_Code...
1,1,SiebDesEratosthenes,./data/filteredData/Participant01/SiebDesErato...
2,1,IsAnagram,./data/filteredData/Participant01/IsAnagram_Co...
3,1,RemoveDoubleChar,./data/filteredData/Participant01/RemoveDouble...
4,1,BinToDecimal,./data/filteredData/Participant01/BinToDecimal...
...,...,...,...
1067,71,BogoSort,./data/filteredData/Participant71/BogoSort_Cod...
1068,71,ReverseQueue,./data/filteredData/Participant71/ReverseQueue...
1069,71,Ackerman,./data/filteredData/Participant71/Ackerman_Cod...
1070,71,RabbitTortoise,./data/filteredData/Participant71/RabbitTortoi...


In [4]:
df_filtered = pd.read_csv("./data/filteredData/filtered_data.csv")
Algorithms = df_filtered["Algorithm"].unique()
Participants = df_filtered[df_filtered["IsOutlier"] == False]["Participant"].unique()

In [5]:
def doBoxesCollide(a, b):
    a_x_center = a[0]
    a_y_center = a[1]
    a_width = a[2]
    a_height = a[3]
    b_x_center = b[0]
    b_y_center = b[1]
    b_width = b[2]
    b_height = b[3]
    return abs(a_x_center - b_x_center) * 2 < (a_width + b_width) and abs(a_y_center - b_y_center) * 2 < (a_height + b_height)

# AOI based analysis

In [6]:
# Get AOI based AOIs
df_aois = pd.DataFrame(columns=["Algorithm", "AOI", "AOIIdx", "BoundingBox"])
for snippet in tqdm(Algorithms):
    aoi_aoi_generator = f"./../CodeSnippets/Generators_Labeled/Generators/{snippet}.json"
    try:
        image, aoi_list = gsl.create_image(aoi_aoi_generator, font_path="./../CodeSnippets/fonts/ttf/")
    except:
        print(f"{snippet} failed")
        image, aoi_list = gsl.create_image(aoi_aoi_generator, font_path="./../CodeSnippets/fonts/ttf/", logging=True)
        break
    height, width = image.size
    width_offset = int(1920 * 0.5) - int(height / 2)
    height_offset = int(1080 * 0.5) - int(width / 2)
    aoi_clustered = []
    current_left = []
    current_top = []
    current_right = []
    current_bottom = []
    current_aoi = []
    color = []
    for letter in aoi_list:
        # Close AOI
        if letter['letter'] in " \t\n":
            continue
        if len(letter["AOI"]) == 1:
            for idx in range(len(current_aoi)):
                aoi_clustered.append((len(aoi_clustered), current_aoi[idx], current_left[idx], current_top[idx], current_right[idx], current_bottom[idx], color[idx]))

            current_aoi = []
            color = []
            current_left = []
            current_top = []
            current_right = []
            current_bottom = []
            continue
        # There is no AOI set
        if len(current_aoi) == 0:
            current_aoi = []
            color = []
            current_left = []
            current_top = []
            current_right = []
            current_bottom = []
            for idx in range(1, len(letter["AOI"])):
                current_aoi.append(letter["AOI"][idx])
                current_left.append(letter["BoundingBox"][0])
                current_top.append(letter["BoundingBox"][1])
                current_right.append(letter["BoundingBox"][2])
                current_bottom.append(letter["BoundingBox"][3])
                color.append(letter["color"])
            continue


        for idx in reversed(range(len(current_aoi))):
            if current_aoi[idx] in letter["AOI"]:
                current_left[idx] = min(current_left[idx], letter["BoundingBox"][0])
                current_top[idx] = min(current_top[idx], letter["BoundingBox"][1])
                current_right[idx] = max(current_right[idx], letter["BoundingBox"][2])
                current_bottom[idx] = max(current_bottom[idx], letter["BoundingBox"][3])
                # remove the AOI from the letter
                letter["AOI"].remove(current_aoi[idx])
            else:
                aoi_clustered.append((len(aoi_clustered), current_aoi[idx], current_left[idx], current_top[idx], current_right[idx], current_bottom[idx], color[idx]))
                del current_aoi[idx]
                del current_left[idx]
                del current_top[idx]
                del current_right[idx]
                del current_bottom[idx]
                del color[idx]

        for idx in range(1, len(letter["AOI"])):
            current_aoi.append(letter["AOI"][idx])
            current_left.append(letter["BoundingBox"][0])
            current_top.append(letter["BoundingBox"][1])
            current_right.append(letter["BoundingBox"][2])
            current_bottom.append(letter["BoundingBox"][3])
            color.append(letter["color"])

    for idx in range(len(current_aoi)):
        aoi_clustered.append((len(aoi_clustered), current_aoi[idx], current_left[idx], current_top[idx], current_right[idx], current_bottom[idx], color[idx]))

    for aoi in aoi_clustered:
        df_aois.loc[len(df_aois)] = [snippet, aoi[1], aoi[0],
                                     (aoi[2] + width_offset,
                                      aoi[3] + height_offset,
                                      aoi[4] + width_offset,
                                      aoi[5] + height_offset)]

df_aois

  0%|          | 0/32 [00:00<?, ?it/s]

,Algorithm,AOI,AOIIdx,BoundingBox
0,IsPrime,method_identifier,0,"(993, 444, 1070, 460)"
1,IsPrime,method_signature,1,"(751, 444, 1202, 464)"
2,IsPrime,method_argument_declaration,2,"(1070, 444, 1202, 463)"
3,IsPrime,for_head,3,"(795, 469, 1147, 489)"
4,IsPrime,arithmetic_expression,4,"(883, 494, 994, 510)"
...,...,...,...,...
626,Rectangle,method_call_identifier,14,"(1048, 769, 1114, 789)"
627,Rectangle,method_arguments,15,"(1114, 769, 1136, 788)"
628,Rectangle,method_call,16,"(993, 769, 1136, 789)"
629,Rectangle,arithmetic_expression,17,"(828, 769, 1136, 789)"


### Check which Fixation of which Participant is in which AOI

In [22]:
df_aoi_fixation_per_participant = pd.DataFrame([], columns=["Algorithm", "Participant", "FixationNumber","Start", "End", "AOIIdx"])
for snippet in tqdm(Algorithms):
    df_aois_per_algo = df_aois[df_aois["Algorithm"] == snippet]

    for participant in Participants:

        df_grouped = df_eyetracking_events[(df_eyetracking_events["Algorithm"] == snippet) & (df_eyetracking_events["Participant"] == participant)]

        if len(df_grouped) == 0:
            continue

        eyetracking_path = df_grouped["Path"].values[0]
        df_current_eyetracking = pd.read_csv(eyetracking_path)
        label = df_current_eyetracking["label"].unique()
        df_fix = df_current_eyetracking[df_current_eyetracking["label"] == "FIXA"]
        df_fix = df_fix.reset_index()
        df_fix["duration"] = df_fix["end_time"] - df_fix["start_time"]
        df_fix["x_range"] = (df_fix["end_x"] - df_fix["start_x"]).apply(abs)
        df_fix["y_range"] = (df_fix["end_y"] - df_fix["start_y"]).apply(abs)
        for fix_idx, fix_row in df_fix.iterrows():
            rectangle_a = [fix_row["start_x"], fix_row["start_y"], fix_row["x_range"], fix_row["y_range"]]
            was_in_aoi = False
            for _, aoi_row in df_aois_per_algo.iterrows():
                aoi_idx = aoi_row["AOIIdx"]
                aoi_name = aoi_row["AOI"]
                bounding_box = aoi_row["BoundingBox"]
                rectangle_b = [(bounding_box[0] + bounding_box[2])/ 2, (bounding_box[1] + bounding_box[3])/ 2,
                               bounding_box[2] - bounding_box[0], bounding_box[3] - bounding_box[1]]
                # check if any value is true
                if doBoxesCollide(rectangle_a, rectangle_b):
                    df_aoi_fixation_per_participant.loc[len(df_aoi_fixation_per_participant)] = [snippet, participant, fix_idx, fix_row["start_time"], fix_row["end_time"], aoi_idx]
                    was_in_aoi = True
            if not was_in_aoi:
                    df_aoi_fixation_per_participant.loc[len(df_aoi_fixation_per_participant)] = [snippet, participant, fix_idx, fix_row["start_time"], fix_row["end_time"], None]

  0%|          | 0/32 [00:00<?, ?it/s]

In [32]:
# save the result
df_aois_tmp = df_aois.copy()
df_aois_tmp = df_aois_tmp[["Algorithm", "AOIIdx", "AOI"]]
for algorithm in df_aois_tmp["Algorithm"].unique():
    df_aois_tmp.loc[len(df_aois_tmp)] = [algorithm, -1, ""]

df_aoi_fixation_per_participant["AOIIdx"] = df_aoi_fixation_per_participant["AOIIdx"].fillna(-1.0).astype(int)
df_aoi_fixation_per_participant["AOI"] = ""
for idx, row in tqdm(df_aoi_fixation_per_participant.iterrows(), total=len(df_aoi_fixation_per_participant)):
    algorithm = row["Algorithm"]
    aoi_idx = row["AOIIdx"]
    sub_frame = df_aois_tmp[(df_aois_tmp["Algorithm"] == algorithm) & (df_aois_tmp["AOIIdx"] == aoi_idx)]
    if len(sub_frame) != 1:
        continue
    aoi_name = sub_frame["AOI"].values[0]
    df_aoi_fixation_per_participant.loc[idx, "AOI"] = aoi_name
df_aoi_fixation_per_participant.to_csv("./data/fixation_per_participant_per_aoi.csv", index=False)

  0%|          | 0/405200 [00:00<?, ?it/s]

In [7]:
df_aoi_fixation_per_participant = pd.read_csv("./data/fixation_per_participant_per_aoi.csv")
df_aoi_fixation_per_participant = df_aoi_fixation_per_participant.drop("AOI", axis=1)
df_aoi_fixation_per_participant["AOIIdx"] = df_aoi_fixation_per_participant["AOIIdx"].astype(float)
df_aoi_fixation_per_participant.loc[df_aoi_fixation_per_participant["AOIIdx"] < 0, "AOIIdx"] = float("Nan")
df_aoi_fixation_per_participant

,Algorithm,Participant,FixationNumber,Start,End,AOIIdx
0,IsPrime,1,0,0.000,0.228,1.0
1,IsPrime,1,1,0.264,0.440,0.0
2,IsPrime,1,1,0.264,0.440,1.0
3,IsPrime,1,2,0.656,0.736,0.0
4,IsPrime,1,2,0.656,0.736,1.0
...,...,...,...,...,...,...
405195,Rectangle,71,30,8.812,8.944,18.0
405196,Rectangle,71,31,8.996,9.080,18.0
405197,Rectangle,71,32,9.272,9.404,18.0
405198,Rectangle,71,33,9.436,9.596,18.0


### Transform the Data to a Fixation/ Refixation split by Participant

In [8]:
df_aoi_fixation_per_participant = df_aoi_fixation_per_participant[df_aoi_fixation_per_participant["AOIIdx"].isna() == False]

In [9]:
df_aois_current = df_aois.copy()
df_aois_current = df_aois_current.drop(["BoundingBox"], axis=1)
for participant in participants:
    df_aois_current.loc[:, f"AOIFixation_P{participant}"] = df_aois_current.apply(lambda row: [], axis=1)
    df_aois_current.loc[:, f"AOIReFixation_P{participant}"] = df_aois_current.apply(lambda row: [], axis=1)


prev_participant = df_aoi_fixation_per_participant["Participant"].iloc[0]
prev_aoi_idx = df_aoi_fixation_per_participant["AOIIdx"].iloc[0]
prev_algorithm = df_aoi_fixation_per_participant["Algorithm"].iloc[0]
fixations = []
re_fixation = False
for idx, row in tqdm(df_aoi_fixation_per_participant.iterrows(), total=len(df_aoi_fixation_per_participant)):
    participant = row["Participant"]
    aoi_idx = row["AOIIdx"]
    algorithm = row["Algorithm"]
    FixationDuration = row["End"] - row["Start"]

    # fixation switches
    if prev_participant != participant or prev_aoi_idx != aoi_idx:
        index = df_aois_current[(df_aois_current["AOIIdx"] == prev_aoi_idx) & (df_aois_current["Algorithm"] == prev_algorithm)].index[0]
        if re_fixation:
            re_fixations = df_aois_current.loc[index, f"AOIReFixation_P{prev_participant}"]
            re_fixations.extend(fixations.copy())
            df_aois_current.loc[index, f"AOIReFixation_P{prev_participant}"] = re_fixations.copy()
        else:
            df_aois_current.loc[index, f"AOIFixation_P{prev_participant}"] = fixations.copy()
        fixations = []
        # possible new fixation
        re_fixation = False

    sub_frame = df_aois_current[(df_aois_current["AOIIdx"] == aoi_idx) & (df_aois_current["Algorithm"] == algorithm)]
    if len(sub_frame) == 0:
        raise Exception(f"No AOI found for {aoi_idx} in Algorithm {algorithm}")
    len_of_fixation = len(sub_frame[f"AOIFixation_P{participant}"].iloc[0])
    if re_fixation == False and len_of_fixation > 0 and len(fixations) == 0:
        re_fixation = True

    fixations.append(FixationDuration)
    prev_participant = participant
    prev_aoi_idx = aoi_idx
    prev_algorithm = algorithm

  0%|          | 0/341648 [00:00<?, ?it/s]

### Calculate the AOI Based Eyetracking Metrics

In [10]:
# Melt the Dataframe to be thinner so that we have Algorithm aoiIdx and Participant and Keys per Fixation / Refixation
df_aois_current_tmp = df_aois_current.drop(["AOI"], axis=1)
df_aoi_melted = pd.melt(df_aois_current_tmp, id_vars=["Algorithm", "AOIIdx"], var_name="KindOfFixation", value_name="FixationDurations")

# Classify Participant and the Kind and Number of Fixations / Refixations
df_aoi_melted["Participant"] = df_aoi_melted["KindOfFixation"].apply(lambda x: int(x.split("_")[1][1:]))
df_aoi_melted["KindOfFixation"] = df_aoi_melted["KindOfFixation"].apply(lambda x: x.split("_")[0])
df_aoi_melted["KindOfFixation"] = df_aoi_melted["KindOfFixation"].apply(lambda x: "Fixation" if x == "AOIFixation" else "ReFixation")
df_aoi_melted["NumberOfFixations"] = df_aoi_melted["FixationDurations"].apply(lambda x: len(x))

# Get the number of Participants for further calculations
number_of_participants = len(participants)

# Calculate the First Fixation Duration per Participant per AOI
df_aoi_melted["FirstFixationDuration"] = None
df_aoi_melted.loc[df_aoi_melted["KindOfFixation"] == "Fixation" , "FirstFixationDuration"] = df_aoi_melted["FixationDurations"]\
    .apply(lambda x: x[0] if len(x) > 0 else None)

# Calculate the Single Fixation Duration per Participant per AOI
df_aoi_melted["SingleFixationDuration"] = None
df_aoi_melted.loc[df_aoi_melted["KindOfFixation"] == "Fixation" , "SingleFixationDuration"] = df_aoi_melted["FixationDurations"]\
    .apply(lambda x: x[0] if len(x) == 1 else None)

# Calculate the Gaze Duration per Participant per AOI
df_aoi_melted["GazeDuration"] = None
df_aoi_melted.loc[df_aoi_melted["KindOfFixation"] == "Fixation" , "GazeDuration"] = df_aoi_melted["FixationDurations"]\
    .apply(lambda x: sum(x) if len(x) > 0 else None)

# Calculate the Total Time per Participant per AOI
df_aoi_melted_total_time = df_aoi_melted.groupby(["Participant", "Algorithm", "AOIIdx"])\
    .agg({"FixationDurations": lambda x: sum(x.values.sum())})
df_aoi_melted_total_time = df_aoi_melted_total_time.rename(columns={"FixationDurations": "TotalTime"})

# Merge the Dataframes
df_aoi_melted = pd.merge(df_aoi_melted, df_aoi_melted_total_time, on=["Participant", "Algorithm", "AOIIdx"], how="left")

# Cast the Dataframes to the right datatype
df_aoi_melted["FirstFixationDuration"] = df_aoi_melted["FirstFixationDuration"].astype(float)
df_aoi_melted["SingleFixationDuration"] = df_aoi_melted["SingleFixationDuration"].astype(float)
df_aoi_melted["GazeDuration"] = df_aoi_melted["GazeDuration"].astype(float)
df_aoi_melted["TotalTime"] = df_aoi_melted["TotalTime"].astype(float).replace(0, np.nan)

# Read in the Skilllevel
df_skill = pd.read_csv(f"./data/filteredData/filtered_data.csv")
df_skill = df_skill[["Participant", "SkillScore"]]
df_skill = df_skill.drop_duplicates()

# Merge the Dataframes to combine metrics with the Skilllevel
df_metrics_skill = pd.merge(df_aoi_melted, df_skill, on=["Participant"], how="left")

# Helper Methods for the Metrics (2. Define because of nearness of the code)
def get_no_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    # remove every entry from df fixations on ["Algorithm", "aoiIdx"] where there is a refixation6
    df_fixations = df_fixations[df_fixations["NumberOfFixations"] == 0]
    return df_fixations


def get_single_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    df_refixations = df[df["KindOfFixation"] == "ReFixation"]
    df_refixations = df_refixations[df_refixations["NumberOfFixations"] > 0]
    # remove every entry from df fixations on ["Algorithm", "aoiIdx"] where there is a refixation6
    df_fixations = df_fixations[~df_fixations["AOIIdx"].isin(df_refixations["AOIIdx"].values)]
    # remove every entry from df fixations on where Number Of Fixations is not 1
    df_fixations = df_fixations[df_fixations["NumberOfFixations"] == 1]
    return df_fixations

def get_multiple_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    df_refixations = df[df["KindOfFixation"] == "ReFixation"]
    df_refixations = df_refixations[df_refixations["NumberOfFixations"] > 0]
    # remove every entry from df fixations on ["Algorithm", "aoiIdx"] where there is a refixation6
    df_fixations = df_fixations[(df_fixations["AOIIdx"].isin(df_refixations["AOIIdx"].values)) & (df_fixations["NumberOfFixations"] >= 1)]
    return df_fixations

def get_fixations(df):
    df_fixations = df[df["KindOfFixation"] == "Fixation"]
    # remove every entry from df fixations on where no Fixation is found
    df_fixations = df_fixations[df_fixations["NumberOfFixations"] >= 1]
    return df_fixations



# dataframe for number of fixations per participant
number_of_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])["NumberOfFixations"].count()
number_of_fixation_per_algorithm = number_of_fixation_per_algorithm.reset_index()

# dataframe for number of aois per algorithm
number_of_aois_per_algorithm = df_metrics_skill.groupby(["Algorithm"])["AOIIdx"].max()
number_of_aois_per_algorithm = number_of_aois_per_algorithm.reset_index()

# dataframe for number of aois with no fixation per algorithm per participant
df_no_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_no_fixations)\
    .drop(["Algorithm"], axis=1)

# dataframe for number of aois with only one fixation per algorithm per participant
df_single_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_single_fixations)\
    .drop(["Algorithm"], axis=1)

# dataframe for number of aois with more than one fixation per algorithm per participant
df_multiple_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_multiple_fixations)\
    .drop(["Algorithm"], axis=1)

# dataframe for number of aois with more or equal than one fixation per algorithm per participant
df_fixation_per_algorithm = df_metrics_skill.groupby(["Participant", "Algorithm"])\
    .apply(get_fixations)\
    .drop(["Algorithm"], axis=1)

# Reformat the dataframes
no_fixation_per_algorithm = df_no_fixation_per_algorithm[["AOIIdx"]]
no_fixation_per_algorithm = no_fixation_per_algorithm.reset_index().drop(["level_2"], axis=1)

single_fixation_per_algorithm = df_single_fixation_per_algorithm[["AOIIdx"]]
single_fixation_per_algorithm = single_fixation_per_algorithm.reset_index().drop(["level_2"], axis=1)

multiple_fixation_per_algorithm = df_multiple_fixation_per_algorithm[["AOIIdx"]]
multiple_fixation_per_algorithm = multiple_fixation_per_algorithm.reset_index().drop(["level_2"], axis=1)

fixations_per_algorithm = df_fixation_per_algorithm[["AOIIdx"]]
fixations_per_algorithm = fixations_per_algorithm.reset_index().drop(["level_2"], axis=1)

# Helper Method for Probability Metrics
def group_len_divided_by_number(current_df, counting_df):
    algorithm = current_df["Algorithm"].iloc[0]
    number_of_aois = counting_df[counting_df["Algorithm"] == algorithm]["AOIIdx"].iloc[0]
    value = len(current_df) / number_of_aois
    return len(current_df) / (number_of_aois + 1)

# Calculate the Metrics per Participant
# Probability of no fixation
df_no_fixation_probability = no_fixation_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_aois_per_algorithm))
df_no_fixation_probability = df_no_fixation_probability.reset_index()

# Probability of single fixation
df_single_fixation_probability = single_fixation_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_aois_per_algorithm))
df_single_fixation_probability = df_single_fixation_probability.reset_index()

# Probability of multiple fixation
df_multiple_fixation_probability = multiple_fixation_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_aois_per_algorithm))
df_multiple_fixation_probability = df_multiple_fixation_probability.reset_index()

# Probability of fixation
df_fixation_probability = fixations_per_algorithm.groupby(["Participant", "Algorithm"]).apply(lambda df: group_len_divided_by_number(df, number_of_aois_per_algorithm))
df_fixation_probability = df_fixation_probability.reset_index()

# Calculate the means for the metrics per algorithm
df_no_fixation_probability = df_no_fixation_probability.groupby(["Participant"]).mean()
df_single_fixation_probability = df_single_fixation_probability.groupby(["Participant"]).mean()
df_multiple_fixation_probability = df_multiple_fixation_probability.groupby(["Participant"]).mean()
df_fixation_probability = df_fixation_probability.groupby(["Participant"]).mean()

# Raw Durations Metrics
# Duration of first fixation
df_first_fixation = df_metrics_skill[~df_metrics_skill["FirstFixationDuration"].isnull()]
df_first_fixation = df_first_fixation.groupby(["Participant"])["FirstFixationDuration"].mean()

# Duration of single fixation
df_single_fixation = df_metrics_skill[~df_metrics_skill["SingleFixationDuration"].isnull()]
df_single_fixation = df_single_fixation.groupby(["Participant"])["SingleFixationDuration"].mean()

# Duration of gaze duration
df_gaze_duration = df_metrics_skill[~df_metrics_skill["GazeDuration"].isnull()]
df_gaze_duration = df_gaze_duration.groupby(["Participant"])["GazeDuration"].mean()

# Total Time
df_total_time = df_metrics_skill[~df_metrics_skill["TotalTime"].isnull()]
df_total_time = df_total_time.groupby(["Participant"])["TotalTime"].mean()

# Put every metric dataframe together into one
df_combined = pd.DataFrame({"FirstFixationDuration": df_first_fixation.values,
                            "SingleFixationDuration": df_single_fixation.values,
                            "GazeDuration": df_gaze_duration.values,
                            "TotalTime": df_total_time.values,
                            "AOINoFixationProbability": df_no_fixation_probability.values.reshape(37, ),
                            "AOISingleFixationProbability": df_single_fixation_probability.values.reshape(37, ),
                            "AOIMultipleFixationProbability": df_multiple_fixation_probability.values.reshape(37, ),
                            "AOIFixationProbability": df_fixation_probability.values.reshape(37, ),
                            "Skill": df_metrics_skill.groupby(["Participant"])["SkillScore"].mean().values})
# get spearman correlation for metrics and skill level
df_combined.corrwith(df_metrics_skill["SkillScore"])

FirstFixationDuration            -1.523017e-15
SingleFixationDuration            7.221338e-17
GazeDuration                     -4.093485e-17
TotalTime                        -2.761301e-16
AOINoFixationProbability         -1.947696e-16
AOISingleFixationProbability      1.609461e-16
AOIMultipleFixationProbability    1.953754e-15
AOIFixationProbability           -1.023443e-15
Skill                            -2.225634e-16
dtype: float64

In [11]:
df_combined.describe()

,FirstFixationDuration,SingleFixationDuration,GazeDuration,TotalTime,AOINoFixationProbability,AOISingleFixationProbability,AOIMultipleFixationProbability,AOIFixationProbability,Skill
count,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000
mean,0.154036,0.153855,0.179826,3.859621,0.349632,0.135855,0.713390,0.799268,0.325230
std,0.026215,0.026328,0.037157,1.674511,0.140566,0.023626,0.106654,0.104031,0.102510
min,0.106272,0.106149,0.112420,1.408802,0.112327,0.094719,0.335170,0.416149,0.142520
25%,0.134010,0.132343,0.147922,2.817690,0.259988,0.118509,0.675850,0.751800,0.259896
50%,0.150493,0.151062,0.174769,3.610357,0.317744,0.138330,0.710154,0.813765,0.315932
75%,0.173985,0.174368,0.198309,4.638764,0.440017,0.147765,0.789021,0.874982,0.379187
max,0.215400,0.214731,0.280047,9.329145,0.713898,0.195601,0.875634,0.943837,0.656726
